## Interactively running scenaria

In [1]:
import pandas as pd
import os
import pickle

from pipeline.scenario import Scenario
from pipeline.constants import *

In [2]:
scenario_params_grid = [
                        dict(
                            **dates,
                            vaccine_effectiveness=vaccine_effectiveness,
                            daily_vaccine_budget=daily_vaccine_budget,
                            min_allocation_factor=min_allocation_factor,
                            balanced_location=balanced_location
                        )
                        for dates in DATES_GRID
                        for vaccine_effectiveness in VACCINE_EFFECTIVENESS_GRID
                        for daily_vaccine_budget in DAILY_VACCINE_BUDGET_GRID
                        for min_allocation_factor in MIN_ALLOCATION_FACTOR_GRID
                        for balanced_location in BALANCED_LOCATIONS_GRID
                    ]

In [3]:
for i, scenario_params in enumerate(scenario_params_grid):
    
    print("Scenario",i)

    start_date = scenario_params["start_date"]
    end_date = scenario_params["start_date"]
    mortality_rate_path = f"{MORTALITY_RATES_PATH}2021-base.npy"
    reload_mortality_rate = os.path.isfile(mortality_rate_path)

    baseline_obj_val, optimized_obj_val = Scenario(**scenario_params).run(
        model_path=f"{MODEL_PATH_PATH}{i}.pickle",
        baseline_solution_path=f"{BASELINE_SOLUTION_PATH}{i}.pickle",
        optimized_solution_path=f"{OPTIMIZED_SOLUTION_ATH}{i}.pickle",
        mortality_rate_path=mortality_rate_path,
        reload_mortality_rate=reload_mortality_rate
    )
    scenario_params["baseline_obj_val"] = baseline_obj_val
    scenario_params["optimized_obj_val"] = optimized_obj_val

results = pd.DataFrame(scenario_params_grid)
results["abs_improvement"] = results["baseline_obj_val"] - results["optimized_obj_val"]
results["pct_improvement"] = results["abs_improvement"] / results["baseline_obj_val"] * 1e2
results.to_csv(RESULTS_PATH)

Scenario 0
Loading model...
Running baseline...
Optimizing...
Restart: 1/1
Iteration: 0/10 	 Objective value: 146416.61
Academic license - for non-commercial use only - expires 2021-04-02
Using license file /Users/vvdig/gurobi.lic
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 298245 rows, 294735 columns and 1120751 nonzeros
Model fingerprint: 0xb39df858
Variable types: 294184 continuous, 551 integer (500 binary)
Coefficient statistics:
  Matrix range     [4e-09, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+07]
Presolve removed 298164 rows and 294652 columns
Presolve time: 0.95s
Presolved: 81 rows, 83 columns, 330 nonzeros
Variable types: 83 continuous, 0 inte

In [5]:
population = pd.read_csv(POPULATION_DATA_PATH)
states = population['state'].unique()

baseline_solutions = []
optimized_solutions = []

for i in range(len(scenario_params_grid)):
    baseline_solution_path=f"{BASELINE_SOLUTION_PATH}{i}.pickle"
    baseline_solutions.append(pickle.load(open(baseline_solution_path, "rb")))
    optimized_solution_path=f"{OPTIMIZED_SOLUTION_ATH}{i}.pickle"
    optimized_solutions.append(pickle.load(open(optimized_solution_path, "rb")))
    print("\n=====")
    print("Scenario", i)
    print([(optimized_solutions[i].locations[j], states[j]) for j in range(len(states))])
    print(optimized_solutions[i].get_total_deaths(), baseline_solutions[i].get_total_deaths())
    


=====
Scenario 0
[(5.0, 'Alabama'), (1.0, 'Alaska'), (2.0, 'Arizona'), (4.0, 'Arkansas'), (1.0, 'California'), (4.0, 'Colorado'), (2.0, 'Connecticut'), (1.0, 'Delaware'), (1.0, 'District of Columbia'), (5.0, 'Florida'), (1.0, 'Georgia'), (1.0, 'Hawaii'), (1.0, 'Idaho'), (1.0, 'Illinois'), (3.0, 'Indiana'), (2.0, 'Iowa'), (2.0, 'Kansas'), (1.0, 'Kentucky'), (2.0, 'Louisiana'), (1.0, 'Maine'), (1.0, 'Maryland'), (1.0, 'Massachusetts'), (3.0, 'Michigan'), (2.0, 'Minnesota'), (1.0, 'Mississippi'), (2.0, 'Missouri'), (2.0, 'Montana'), (2.0, 'Nebraska'), (1.0, 'Nevada'), (1.0, 'New Hampshire'), (2.0, 'New Jersey'), (2.0, 'New Mexico'), (3.0, 'New York'), (1.0, 'North Carolina'), (1.0, 'North Dakota'), (1.0, 'Ohio'), (2.0, 'Oklahoma'), (2.0, 'Oregon'), (2.0, 'Pennsylvania'), (2.0, 'Rhode Island'), (3.0, 'South Carolina'), (2.0, 'South Dakota'), (2.0, 'Tennessee'), (4.0, 'Texas'), (2.0, 'Utah'), (1.0, 'Vermont'), (2.0, 'Virginia'), (5.0, 'Washington'), (1.0, 'West Virginia'), (2.0, 'Wisconsin

array([  233250.80292789,    35181.36220916,  1225297.99760839,
         525082.74633074,   163430.94729732,  2637757.18879408,
         170478.26681616,    19574.42566359,    80074.93945976,
        4719515.96004988,        0.        ,    67768.1491676 ,
              0.        ,   546676.12987018,   825574.26905787,
         580338.15564963,    46173.62972398,        0.        ,
        1000476.10889058,    63851.97929343,  1503921.04528963,
         940838.59553878,    44533.12597336,   132921.13742578,
         573124.15337865,   867825.6790302 ,    46150.63812361,
         136830.77398162,        0.        ,        0.        ,
        2528100.17751906,   394498.63213657,  4100888.11447074,
              0.        ,    31220.533311  ,        0.        ,
         591039.19963184,  1197533.10981788,  1247639.21663478,
          53802.16735045,   658996.17269271,    45616.45741362,
         231356.87130904, 13964397.25956678,   640485.70316587,
         195467.65483661,   408076.04102

In [6]:
V = optimized_solutions[0].vaccinated
print(np.nansum(V), V.shape[2]*0.6e6)

for t in range(V.shape[2]):
    print(V[:, :, t].sum())

46326656.36764933 53400000.0
600000.0
600000.0
595311.8329368279
597196.2776965618
573820.8694901469
600000.0
537136.8383358882
596873.4755482187
560332.1516730294
512224.38351731375
588955.803713439
509142.8316008609
591423.157728414
600000.0
472777.3352272806
559935.6253564404
565487.8940248364
470637.727188812
593403.3066441554
556329.8119568827
555127.8428991509
518271.2145162376
509766.74203292676
581911.9278541749
555601.7576855975
545705.4442850298
569533.0906609413
541717.9431462621
461519.39135514025
584780.6325196545
545230.2167012342
589441.712562039
599803.957998159
474708.7047054182
446685.16688990744
549718.8902858078
547223.3784074074
574597.23016669
597974.4124548701
577321.0888176674
554810.3850219973
479032.0373541369
521922.56015534577
465626.8983364323
553485.4049470354
594611.0560476425
564958.1336860919
506170.6893837225
600000.0
444897.3289570039
493985.56541528436
536757.6837761726
559233.7224404011
542202.3468336852
524057.9413140119
570658.0531513595
599574.50